In [1]:
import tensorflow as tf

import energy_py
from energy_py.agents import DQN, tfValueFunction
from energy_py.envs import BatteryEnv

Using TensorFlow backend.


In [2]:
EPISODE_LENGTH = 5
BRAIN_PATH = '/'
BATCH_SIZE = 10
EPISODES = 10
DISCOUNT = 0.9

total_steps = EPISODES * EPISODE_LENGTH

In [3]:
path = '/home/adam/git/energy_py/energy_py/experiments/battery/'
path_w = '/mnt/c/Users/Adam Green/Documents/GitHub/energy_py/energy_py/experiments/battery/'

env = BatteryEnv(data_path=path,
                 episode_length=10)

In [4]:
agent = DQN(env, 
            DISCOUNT, 
            brain_path='/',
            Q=tfValueFunction,
            batch_size=BATCH_SIZE,
            total_steps=total_steps)

In [7]:

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    #  initialize before starting episode
    done, step, total_step = False, 0, 0
    episode = 1
    observation = env.reset(episode)

    #  while loop runs through a single episode
    while done is False:
        #  select an action
        action = agent.act(sess=sess, obs=observation)
        #  take one step through the environment
        next_observation, reward, done, info = env.step(action)
        #  store the experience
        agent.memory.add_experience(observation, action, reward,
                                    next_observation, done, step, episode)
        step += 1
        total_step += 1
        observation = next_observation
        
        batch = agent.memory.get_random_batch(BATCH_SIZE)

        if step == 5:
            #  with DQN we can learn within episode 
            #  get a batch to learn from
            batch = agent.memory.get_random_batch(BATCH_SIZE)

            train_info = agent.learn(sess=sess, batch=batch)
            agent.update_target_net = 1
            if total_step % agent.update_target_net == 0:
                agent.update_target_network(sess)


shapes of arrays used in learning
obs shape (10, 6)
actions shape (10, 2)
rews shape (10, 1)
terminal shape (10,)
[False False False False False False False False False False]
max_q after termnal [[ 0.63329256]
 [ 0.44097924]
 [ 0.63329256]
 [ 0.44097924]
 [ 0.63329256]
 [ 0.63329256]
 [ 0.63329256]
 [ 0.3991617 ]
 [ 0.24761711]
 [ 0.63329256]]
before bellman eqn
rews shape (10, 1)
max_q shape (10, 1)
targets shape (10, 1)
action indicies shape (10,)
inputs shape (10, 6)
Q preds for next_obs shape (10, 100)
max q values has shape (10, 1)
target shape after bellman equn (10,)
target shape processing (10,)


In [ ]:
step

In [ ]:
import numpy as np

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    obs = np.concatenate(batch[:, 0]).reshape(-1, agent.observation_space.shape[0])
    actions = np.concatenate(batch[:, 1]).reshape(-1, agent.action_space.shape[0])
    rews = batch[:, 2].reshape(-1, 1)
    next_obs = np.concatenate(batch[:, 3]).reshape(-1, agent.observation_space.shape[0])
    terminal = np.array(batch[:, 4], dtype=np.bool).flatten()
    
    inputs = agent.state_processor.transform(obs)
    next_obs = agent.state_processor.transform(next_obs)
    preds = agent.Q_target.predict(sess, next_obs)
    max_q = np.max(preds, axis=1).reshape(preds.shape[0],-1)
    
    max_q[terminal] = 0
    
    targets = rews + 0.9 * max_q
    
    act_list = agent.actions.tolist()
    action_index = [act_list.index(act) for act in actions.tolist()]
    action_index = np.array(action_index).reshape(-1)
    print(preds.shape)
    print(action_index.shape)
    rng = np.arange(preds.shape[0])
#     t_ = tf.placeholder(tf.float32, [None, 1], 'target')
        
#     act_i = tf.placeholder(tf.int32, [None, 2], 'selected_act')
    
#     preds_ = tf.placeholder(tf.float32, [None, preds.shape[1]], 'preds')
    
    x = tf.placeholder('float32', (None, None), 'x')
    idx1 = tf.placeholder('int32', (None), 'in1')
    idx2 = tf.placeholder('int32', (None), 'in2')
    
    res = tf.gather_nd(x, tf.stack((idx1, idx2), -1))
    
    out = sess.run(res, {x:preds,
                        idx1: rng,
                        idx2: action_index})

In [ ]:
out.shape

In [ ]:
action_index

In [ ]:
with tf.Session() as sess:
        
    x = tf.constant([[1, 2, 3],
                 [4, 5, 6],
                 [7, 8, 9]])
    idx = tf.constant([1, 0, 2])
    
    y = tf.gather(x,  # flatten input
                  idx)  # use flattened indices
    out = sess.run(y)

In [ ]:
out

In [ ]:
test = np.arange(10).reshape(-1,2)
out = np.concatenate([test, test], axis=1)

In [ ]:
out